# Practical: building an economic statistics database

The goal here is to have a bi-temporal database of interesting economic statistics (e.g. gross domestic product,
consumer price index, U-6 employment rate.) With that, we can build models of the economy, interesting
visualizations, etc.

The source of the data with be FRED/ALFRED: the Federal Reserve of St. Louis's online db.
    
The plan is to query ALFRED for web data as needed, then store in our database in form that works nicely 
within our framework. I alost feel bad doing this because the source is so nice and so close to being bi-temporal
that it would be almost reasonable to declare ALFRED a system-of-record in whatever we are building: making copies
of the data seems to show somewhat of a lack of trust. But...

1. I want my apps to run even if the internet is down.
2. I want 100% report reproducibility even if ALFRED breaks a bi-temporal promise (by accident,
funding constraints, a legal mandate, etc.)
3. ALFRED timeseries data is bi-temporal, but the meta-data (e.g. categories) does not seem to be.
4. Our bi-temporality is at the UTC microsecond level, ALFRED seems at the daily, naked date level. It's
unclear how to even map between the two.

So, in our space, bi-temporal time is just:
* transaction - when we record our copy of some ALFRED data.
* valid - either just a copy of physical, but maybe the ALFRED update time if we can understand and trust that.

We may also add an additional time-line ('ALFRED') that supports transaction/valid dates in the ALFRED world.
That way, for example, calls such as *consumerPriceIndex.value()* would behave in a time-aware fashion.

In [1]:
import mand.core

from mand.core import Entity, node, Context, _tr, ObjectDb, displayDict, displayListOfDicts
from mand.core import PrintMonitor, SummaryMonitor

db = ObjectDb()

In [2]:
import os
import sys
import json
import xml.etree.ElementTree as ET
if sys.version_info[0] >= 3:
    import urllib.request as url_request
    import urllib.parse as url_parse
    import urllib.error as url_error
else:
    import urllib2 as url_request
    import urllib as url_parse
    import urllib2 as url_error

urlopen = url_request.urlopen
quote_plus = url_parse.quote_plus
urlencode = url_parse.urlencode
HTTPError = url_error.HTTPError

In [3]:
class FredManager(Entity):
    # a hacked version of somebody else's code.
    
    max_results_per_request = 1000
    
    @node
    def rootUrl(self):
         return 'https://api.stlouisfed.org/fred'
        
    @node
    def apiKeyFileName(self):
        return '../fred/.api_key'
    
    @node
    def apiKey(self):
        api_key_file = self.apiKeyFileName()
        f = open(api_key_file, 'r')
        ret = f.readline().strip()
        return ret
    
    def __fetch_data(self, url):
        url += '&api_key=' + self.apiKey()
        try:
            response = urlopen(url)
            root = ET.fromstring(response.read())
        except HTTPError as exc:
            root = ET.fromstring(exc.read())
            raise ValueError(root.get('message'))
        return root
    
    def __fetch_data_json(self, url):
        url += '&api_key=' + self.apiKey()
        url += '&file_type=json'
        try:
            response = urlopen(url)
            r = response.read()
        except HTTPError as exc:
            root = ET.fromstring(exc.read())
            raise ValueError(root.get('message'))
        ret = json.loads(r)
        return ret

    
    def get_series_info(self, series_id):
        url = "%s/series?series_id=%s" % (self.rootUrl(), series_id)
        root = self.__fetch_data(url)
        if root is None or not len(root):
            raise ValueError('No info exists for series id: ' + series_id)
        info = root[0].attrib
        return info
    
    def get_series_all_releases(self, series_id):
        earliest_realtime_start = '1776-07-04'
        latest_realtime_end = '9999-12-31'
        f = "%s/series/observations?series_id=%s&realtime_start=%s&realtime_end=%s"
        url =  f % (self.rootUrl(),
                    series_id,
                    earliest_realtime_start,
                    latest_realtime_end)
                                                                                        
        root = self.__fetch_data(url)
        if root is None:
            raise ValueError('No data exists for series id: ' + series_id)
            
        data = [ child.attrib for child in root ]
        return data

    def search_category(self, category_id=0):
        url = "%s/category?category_id=%s&" % (self.rootUrl(), category_id)
        return self.__fetch_data_json(url)
        
    def search_category_children(self, category_id=0):
        url = "%s/category/children?category_id=%s&" % (self.rootUrl(), category_id)
        return self.__fetch_data_json(url)
        
    def __do_series_search(self, url):
        root = self.__fetch_data(url)
        series_ids = []
        num_results_returned = 0  # number of results returned in this HTTP request
        num_results_total = int(root.get('count'))  # total number of results, this can be larger than number of results returned
        for child in root:
            num_results_returned += 1
            series_id = child.get('id')
            series_ids.append(series_id)
        return series_ids, num_results_total
    
    def __get_search_results(self, url):
        data, num_results_total = self.__do_series_search(url)
        if data is None:
            return data

        max_results_needed = num_results_total

        if max_results_needed > self.max_results_per_request:
            for i in range(1, max_results_needed // self.max_results_per_request + 1):
                offset = i * self.max_results_per_request
                next_data, _ = self.__do_series_search(url + '&offset=' + str(offset))
                data.append(next_data)
        return data
    
    def search_by_category(self, category_id):
        url = "%s/category/series?category_id=%s&" % (self.rootUrl(), category_id)
        info = self.__get_search_results(url)
        if info is None:
            raise ValueError('No series exists for category id: ' + str(category_id))
        return info
    
_tr.add(FredManager)
fm = _tr.FredManager('Main', db=db).write()

In [4]:
class FredSeries(Entity):
    @node 
    def fredManager(self):
        return _tr.FredManager.get('Main', db=self.meta.db)
    
    @node
    def name(self):
        return self.meta.name()
    
    @node
    def info(self):
        fm = self.fredManager()
        return fm.get_series_info(self.name())
    
    @node
    def allReleases(self):
        return fm.get_series_all_releases(self.name())
    
    """
    ['observation_end', 'last_updated', 'observation_start', 'title', 'seasonal_adjustment_short', 
     'seasonal_adjustment', 'notes', 'popularity', 'realtime_end', 'frequency', 'units_short', 
     'units', 'realtime_start', 'id', 'frequency_short']
    """

In [5]:
class FredCategory(Entity):
    @node 
    def fredManager(self):
        return _tr.FredManager.get('Main', db=self.meta.db)
    
    @node
    def id(self):
        return self.meta.name()
    
    @node
    def info(self):
        return fm.search_category(self.id())
    
    @node
    def name(self):
        info = self.info()
        return info['categories'][0]['name']
    
    @node
    def children(self):
        ret = fm.search_category_children(self.id())
        return ret.get('categories')
        
    @node
    def seriesNames(self):
        return fm.search_by_category(self.id())

In [6]:
with db:
    cpi = FredSeries('CPIAUCSL')

displayDict( cpi.info())

|key|value|
|-|-|
|observation_end|2017-01-01
|last_updated|2017-02-15 07:51:14-06
|observation_start|1947-01-01
|title|Consumer Price Index for All Urban Consumers: All Items
|seasonal_adjustment_short|SA
|seasonal_adjustment|Seasonally Adjusted
|notes|The Consumer Price Index for All Urban Consumers: All Ite...
|popularity|100
|realtime_end|2017-03-12
|frequency|Monthly
|units_short|Index 1982-1984=100
|units|Index 1982-1984=100
|realtime_start|2017-03-12
|id|CPIAUCSL
|frequency_short|M

In [7]:
with db:
    gdp = FredSeries('GDP')
    
displayDict(gdp.info())

|key|value|
|-|-|
|observation_end|2016-10-01
|last_updated|2017-02-28 07:51:17-06
|observation_start|1947-01-01
|title|Gross Domestic Product
|seasonal_adjustment_short|SAAR
|seasonal_adjustment|Seasonally Adjusted Annual Rate
|notes|BEA Account Code: A191RC1  Gross domestic product (GDP), ...
|popularity|89
|realtime_end|2017-03-12
|frequency|Quarterly
|units_short|Bil. of $
|units|Billions of Dollars
|realtime_start|2017-03-12
|id|GDP
|frequency_short|Q

In [8]:
r = gdp.allReleases()
print len(r)
for i in range(2):
    displayDict(r[i])
displayListOfDicts(r[1000:1020])


2303


|key|value|
|-|-|
|date|1946-01-01
|realtime_start|1992-12-22
|realtime_end|1996-01-18
|value|199.7

|key|value|
|-|-|
|date|1946-01-01
|realtime_start|1996-01-19
|realtime_end|1997-05-06
|value|.

|date|realtime_end|realtime_start|value|
|-|-|-|-|
|1979-04-01|1997-05-06|1996-01-19|2522.4
|1979-04-01|1999-10-27|1997-05-07|2522.3
|1979-04-01|1999-10-28|1999-10-28|.
|1979-04-01|2003-12-09|1999-10-29|2529.3
|1979-04-01|2009-07-30|2003-12-10|2527.6
|1979-04-01|2013-07-30|2009-07-31|2526.4
|1979-04-01|9999-12-31|2013-07-31|2595.9
|1979-07-01|1996-01-18|1991-12-04|2523.3
|1979-07-01|1997-05-06|1996-01-19|2592.6
|1979-07-01|1999-10-27|1997-05-07|2592.8
|1979-07-01|1999-10-28|1999-10-28|.
|1979-07-01|2003-12-09|1999-10-29|2601.5
|1979-07-01|2009-07-30|2003-12-10|2600.7
|1979-07-01|2013-07-30|2009-07-31|2599.7
|1979-07-01|9999-12-31|2013-07-31|2670.4
|1979-10-01|1996-01-18|1991-12-04|2578.8
|1979-10-01|1997-05-06|1996-01-19|2650.1
|1979-10-01|1999-10-27|1997-05-07|2650.4
|1979-10-01|1999-10-28|1999-10-28|.
|1979-10-01|2003-12-09|1999-10-29|2663.8

In [9]:
"""
This seems true, if a little out of date:

Get all data for a Fred series id including first releases and all revisions. 
This returns a DataFrame with three columns: 'date', 'realtime_start', and 'value'. 
For instance, the US GDP for Q4 2013 was first released to be 17102.5 on 2014-01-30, 
and then revised to 17080.7 on 2014-02-28, 
and then revised to 17089.6 on 2014-03-27. 
You will therefore get three rows with the same 'date' (observation date) of 2013-10-01 but three
different 'realtime_start' of 2014-01-30, 2014-02-28, and 2014-03-27 
with corresponding 'value' of 17102.5, 17080.7 and 17089.6
"""

d = '2013-10-01'
t = [ i for i in r if i['date'] == d]
displayListOfDicts(t)

|date|realtime_end|realtime_start|value|
|-|-|-|-|
|2013-10-01|2014-02-27|2014-01-30|17102.5
|2013-10-01|2014-03-26|2014-02-28|17080.7
|2013-10-01|2014-07-29|2014-03-27|17089.6
|2013-10-01|2015-07-29|2014-07-30|17078.3
|2013-10-01|2016-07-28|2015-07-30|16957.6
|2013-10-01|9999-12-31|2016-07-29|16999.9

In [10]:
with db:
    fc = FredCategory('0')
r = fc
for i in range(5):
    print
    print r.name()
    if not r.children():
        break
    displayListOfDicts(r.children())
    r = FredCategory(r.children()[0]['id'])


Categories


|id|name|parent_id|
|-|-|-|
|32991|Money, Banking, & Finance|0
|10|Population, Employment, & Labor Markets|0
|32992|National Accounts|0
|1|Production & Business Activity|0
|32455|Prices|0
|32263|International Data|0
|3008|U.S. Regional Data|0
|33060|Academic Data|0


Money, Banking, & Finance


|id|name|notes|parent_id|
|-|-|-|-|
|22|Interest Rates|None|32991
|15|Exchange Rates|None|32991
|24|Monetary Data|None|32991
|46|Financial Indicators|None|32991
|23|Banking|None|32991
|32360|Business Lending|None|32991
|32145|Foreign Exchange Intervention|In addition to the listed daily intervention series, intr...|32991


Interest Rates


|id|name|parent_id|
|-|-|-|
|33058|Automobile Loan Rates|22
|51|Bankers Acceptance Rate|22
|121|Certificates of Deposit|22
|120|Commercial Paper|22
|32348|Corporate Bonds|22
|33059|Credit Card Loan Rates|22
|32298|Eurodollar Deposits|22
|118|FRB Rates - discount, fed funds, primary credit|22
|33056|Interest Checking Accounts|22
|33446|Interest Rate Spreads|22
|32299|Interest Rate Swaps|22
|33003|LIBOR Rates|22
|32995|Long-Term Securities|22
|33055|Money Market Accounts|22
|114|Mortgage Rates|22
|33057|Personal Loan Rates|22
|117|Prime Bank Loan Rate|22
|33491|Saving Accounts|22
|116|Treasury Bills|22
|115|Treasury Constant Maturity|22
|82|Treasury Inflation-Indexed Securities|22


Automobile Loan Rates


In [11]:
if r.seriesNames():
    for s in r.seriesNames()[:5]:
        series = FredSeries(s, db=fc.meta.db)
        displayDict( series.info())

|key|value|
|-|-|
|observation_end|2011-01-01
|last_updated|2012-06-26 12:01:23-05
|observation_start|1971-06-01
|title|New Car Average Finance Rate at Auto Finance Companies (D...
|seasonal_adjustment_short|NSA
|seasonal_adjustment|Not Seasonally Adjusted
|notes|For further information, please refer to the Board of Gov...
|popularity|45
|realtime_end|2017-03-12
|frequency|Monthly
|units_short|%
|units|Percent
|realtime_start|2017-03-12
|id|TERMAFCNCNSA
|frequency_short|M

|key|value|
|-|-|
|observation_end|2016-11-01
|last_updated|2017-01-09 14:16:21-06
|observation_start|1972-02-01
|title|Finance Rate on Consumer Installment Loans at Commercial ...
|seasonal_adjustment_short|NSA
|seasonal_adjustment|Not Seasonally Adjusted
|notes|For further information, please refer to the Board of Gov...
|popularity|61
|realtime_end|2017-03-12
|frequency|Monthly
|units_short|%
|units|Percent
|realtime_start|2017-03-12
|id|TERMCBAUTO48NS
|frequency_short|M